In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from acquire import get_titanic_data
from prepare import train_validate_test_split

df = get_titanic_data()
df = df.drop(columns='deck')
df = df[~ df.age.isna()]
df = df[~ df.embarked.isna()]

train, validate, test = train_validate_test_split(df)

NameError: name 'train_test_split' is not defined

In [ ]:
df.head()

### Explore the target
- What are we trying to predict?

In [ ]:
#what would a cause of someone surviving the Titanic be??
df.survived.value_counts()

In [ ]:
df.survived.hist()

In [ ]:
survival_rate = df.survived.mean()